# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [18]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

from config import g_key

# Import API key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [19]:
# Load the csv exported in Part I to a DataFrame
weather_df = pd.read_csv("cities.csv")
weather_df.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [20]:
# Store latitude and longitude in locations
locations = weather_df[["Lat","Lng"]]

# Filla NaN values
locations = locations.dropna()
locations.head()


,Lat,Lng
0,68.05,39.51
1,-20.41,57.70
2,77.48,-69.36
3,40.71,112.04
4,-33.93,18.42


In [21]:
# Defining Humidity as weight
weights = weather_df["Humidity"].astype(float)


In [22]:
#Set up your map
fig = gmaps.figure()
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights))
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [23]:
# Narrow new DF to the 3 criterias defined for the exercise
weather2 = weather_df
weather2 = weather2[(weather2["Max Temp"] > 70)]
weather2 = weather2[(weather2["Max Temp"] < 80)] 
weather2 = weather2[(weather2["Wind Speed"] < 10)]
weather2 = weather2[(weather2["Cloudiness"] == 0)]

# drop null
weather2 = weather2.dropna()
weather2.count()

City_ID       7
City          7
Cloudiness    7
Country       7
Date          7
Humidity      7
Lat           7
Lng           7
Max Temp      7
Wind Speed    7
dtype: int64

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [24]:
# Store into variable named hotel_df.
hotel_df = weather2

# set up additional column to hold information
hotel_df['Hotel Name'] = " "
hotel_df.head()

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
70,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32,
88,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50,
138,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06,
139,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59,
176,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93,


In [25]:
# find the closest hotel of each type to coordinates
params = {
    "radius": 5000,
    "type": "hotel",
    "keyword": "hotel",
    "key": g_key,}

# use iterrows to iterate through pandas dataframe
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    
    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    response = requests.get(base_url, params=params)

    # convert to json
    data = response.json()
    
    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = data["results"][0]["name"]
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")


In [26]:
hotel_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
70,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32,Hotel Escarpas do Lago
88,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50,Hotel Castelo
138,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06,MAEVA HOTEL
139,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59,Dandeli Dreams Jungle Resort
176,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93,Hôtel Babel
297,297,mogok,0,MM,1558378844,51,22.92,96.51,72.24,2.77,King Bridge
444,444,birjand,0,IR,1558378880,26,32.86,59.22,71.60,9.17,Mountain Hotel


In [27]:
# Add marker layer ontop of heat map
fig = gmaps.figure(layout={
        'padding': '3px',
        'border': '1px solid black'
})
locations2 = hotel_df[["Lat", "Lng"]]
fig.add_layer(gmaps.heatmap_layer(locations2))
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
hotel_df = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
# Display Map
fig

Figure(layout=FigureLayout(border='1px solid black', height='420px', padding='3px'))